# GNN Optimizer
This notebook runs the optimizer for the GNN model. With the use of Weights & Biases the optimal parameters are determined. Weights & Biases runs sweeps with different parameters while tracking the validation loss. With the data obtained from the runs, it can be determined which parameters play the most important roles and what their optimum values are.

In [1]:
pip install torch_geometric==2.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00


In [2]:
import pickle
from sklearn.metrics import r2_score
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim

from torch_geometric.data import Data
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, TAGConv, ChebConv
import networkx as nx
from sklearn.metrics import confusion_matrix

import timeit
start = timeit.default_timer()

## Data pre-processing
Here the train and validation data is loaded and normalized. Since the effect of the parameters on the validation loss plays the main roll, the test datasets are not used here.

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
folder_path = '/content/drive/MyDrive/FLOOD1/GNN_model'
%cd "$folder_path"
from time_GNN_with_WD0WD1 import convert_to_pyg
import GNN_model as gnn

/content/drive/.shortcut-targets-by-id/1R4EdEhifiDD7VD3aCsLqQ20iIMsm-2Cc/FLOOD1/GNN_model


In [6]:
G = nx.grid_2d_graph(64, 64, create_using=nx.DiGraph)
pos = {i:(x+0.5,y+0.5) for i, (x,y) in enumerate(G.nodes())} # Position mapping for grid layout
mapping = dict(zip(G, range(0, G.number_of_nodes())))
G = nx.relabel_nodes(G, mapping)

In [7]:
folder_path = '/content/drive/MyDrive/FLOOD1/raw_datasets/raw_datasets'
%cd "$folder_path"

# importing data and adding to the grid
start_sim = 1
n_sim = 80 # number of available datasets for training and validation
DEMS = [] # storing DEM
WDS = [] # storing WD
for i in tqdm(range(start_sim,start_sim+n_sim)):
    DEM = np.loadtxt(f"{folder_path}/DEM/DEM_{i}.txt")[:,2]
    DEMS.append(DEM)
    WD = np.loadtxt(f"{folder_path}/WD/WD_{i}.txt")
    WDS.append(WD)

train_dataset_DEM, val_dataset_DEM = train_test_split(DEMS, test_size=0.3, random_state=42)

train_dataset_WD, val_dataset_WD = train_test_split(WDS, test_size=0.3, random_state=42)

/content/drive/.shortcut-targets-by-id/1R4EdEhifiDD7VD3aCsLqQ20iIMsm-2Cc/FLOOD1/raw_datasets/raw_datasets


100%|██████████| 80/80 [00:48<00:00,  1.65it/s]


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0,1))
scaler.fit(train_dataset_DEM)
scaled_DEM_train = scaler.transform(train_dataset_DEM)
scaled_DEM_val = scaler.transform(val_dataset_DEM)

scaler = MinMaxScaler((0,1))

def normalize_WD(scaler, WD_data, scaled_DEM, train=False):
    original_shape = np.array(WD_data).shape # store the original shape to get same output shape
    reshaped_WD = np.array(WD_data).reshape(-1, original_shape[-1]) # from 3D to 2D
    if train: # fit only on training data
        scaler.fit(reshaped_WD)
    scaled_WD = scaler.transform(reshaped_WD)
    final_WD = scaled_WD.reshape(original_shape)
    dataset = []
    for i in range(len(final_WD)):
        grid_i = convert_to_pyg(G, pos, scaled_DEM[i], final_WD[i]) # assigning values to the graph nodes
        dataset.append(grid_i)
    if train:
        return scaler, dataset
    return dataset

scaler, train_dataset = normalize_WD(scaler, train_dataset_WD, scaled_DEM_train, train=True)
val_dataset = normalize_WD(scaler, val_dataset_WD, scaled_DEM_val)

## Log in into Weights & Biases
Note that to run this notebook, you need a Weights & Biases account.

In [9]:
!pip install wandb
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00


In [10]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Define sweep
The sweep method chosen is 'random', to make sure many different combinations are tried. This then clearly shows which parameters are more important and what their optimal values are.

Note a combination of high K, high batch size and high number of MLP layers leads to a crash (ran out of CUDA memory). The follow-up runs will then also crash, even the combination of parameters is less computationally expensive. Therefore it is recommended to test the highest computational expensive run first in the model notebook, before running the optimization notebook. It is does not crash in the model notebook, this notebook will also safely run.

In [11]:
# Choose method for configurations
sweep_config = {
    'method': 'random'
    }

# Choose metric for optimization
metric = {
    'name': 'val_loss',
    'goal': 'minimize'
    }
sweep_config['metric'] = metric

# Choose parameters to vary
parameters_dict = {
    'n_layers_MLP': {
        'values': [2, 3, 4]
        },
    'n_layers_GNN': {
        'values': [3, 4, 6, 8]
        },
    'K_tot': {
        'values': [96, 120, 144, 168, 192]
        },
   'hidden_features': {
        'values': [64, 128]
        },
    'convolution_type': {
        'values': ['ChebConv', 'TAGConv', 'GCNConv']
        },
    'batch_size': {
        'values': [8, 16]
        }
    }

sweep_config['parameters'] = parameters_dict

# Define parameters which are not altered during the sweep, but still want to be mentioned
parameters_dict.update({
    'epochs': {
        'value': 200}
    })
print(sweep_config)

{'method': 'random', 'metric': {'name': 'val_loss', 'goal': 'minimize'}, 'parameters': {'n_layers_MLP': {'values': [2, 3, 4]}, 'n_layers_GNN': {'values': [3, 4, 6, 8]}, 'K_tot': {'values': [96, 120, 144, 168, 192]}, 'hidden_features': {'values': [64, 128]}, 'convolution_type': {'values': ['ChebConv', 'TAGConv', 'GCNConv']}, 'batch_size': {'values': [8, 16]}, 'epochs': {'value': 200}}}


In [12]:
# Define sweep id variable
sweep_id = wandb.sweep(sweep_config, project='encoder_decoder_GNN')

Create sweep with ID: kcs55s71
Sweep URL: https://wandb.ai/floods1/encoder_decoder_GNN/sweeps/kcs55s71


## Run the Sweep agent

In [13]:
def train_epoch(model, loader, optimizer, loss_function, device='cpu'):
    model.to(device)
    model.train()
    losses = []

    for batch in loader:
        batch = batch.to(device)
        preds = model(batch)
        loss = loss_function(preds, batch.y)

        losses.append(loss.cpu().detach())

        loss.backward()   # compute the gradients using backpropagation
        optimizer.step()  # update the weights with the optimizer
        optimizer.zero_grad(set_to_none=True)   # reset the computed gradients

    return np.array(losses).mean()

def evaluation(model, loader, loss_function, device='cpu'):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode
    losses = []

    # Remove gradients computations since we are only evaluating and not training
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            preds = model(batch)

            loss = loss_function(preds, batch.y)
            losses.append(loss.cpu())

    return np.array(losses).mean()

# Set training parameters
learning_rate = 0.001

# Create loss function
loss_function = nn.MSELoss()

# This line is used to select GPU to train, if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
      config = wandb.config
      K = config.K_tot // config.n_layers_GNN
      network = gnn.build_network(config.hidden_features, config.n_layers_MLP, config.n_layers_GNN, K, config.convolution_type).to(device)
      optimizer = torch.optim.AdamW(network.parameters(), lr=learning_rate)
      batch_size = config.batch_size

      # Create the training and validation dataloaders to "feed" data to the model in batches
      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
      val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
      best_val = np.inf # start with an infinite high validation loss, so it will be updated with the first computed loss
      best_epoch = 0
      for epoch in range(config.epochs):
          train_loss = train_epoch(network, train_loader, optimizer, loss_function, device=device)
          val_loss = evaluation(network, val_loader, loss_function, device=device)
          if val_loss < best_val: # update the best validation loss
            best_val = val_loss
            best_epoch = epoch
          if epoch - best_epoch > 20: # stop if the validation loss was not improved for more than 20 epochs
            break

      wandb.log({'val_loss': best_val, 'best val at epoch': best_epoch})

In [ ]:
# Start the sweep, count tells how many configurations are going to be done
wandb.agent(sweep_id, function=train, count=100)

wandb: Agent Starting Run: 00q5at94 with config:
wandb: 	K_tot: 168
wandb: 	batch_size: 16
wandb: 	convolution_type: TAGConv
wandb: 	epochs: 200
wandb: 	hidden_features: 64
wandb: 	n_layers_GNN: 4
wandb: 	n_layers_MLP: 3
wandb: Currently logged in as: annika-schalk (floods1). Use `wandb login --relogin` to force relogin


best val at epoch,▁
val_loss,▁
best val at epoch,54
val_loss,0.01758


wandb: Agent Starting Run: e62vm4qa with config:
wandb: 	K_tot: 192
wandb: 	batch_size: 16
wandb: 	convolution_type: ChebConv
wandb: 	epochs: 200
wandb: 	hidden_features: 64
wandb: 	n_layers_GNN: 6
wandb: 	n_layers_MLP: 4


best val at epoch,▁
val_loss,▁
best val at epoch,48
val_loss,0.0156


wandb: Agent Starting Run: ps2sixt8 with config:
wandb: 	K_tot: 96
wandb: 	batch_size: 8
wandb: 	convolution_type: GCNConv
wandb: 	epochs: 200
wandb: 	hidden_features: 128
wandb: 	n_layers_GNN: 8
wandb: 	n_layers_MLP: 4


...

wandb: Agent Starting Run: 7r67b3u4 with config:
wandb: 	K_tot: 96
wandb: 	batch_size: 8
wandb: 	convolution_type: TAGConv
wandb: 	epochs: 200
wandb: 	hidden_features: 64
wandb: 	n_layers_GNN: 6
wandb: 	n_layers_MLP: 2


best val at epoch,▁
val_loss,▁
best val at epoch,12
val_loss,0.02212


wandb: Agent Starting Run: 41f5pkqa with config:
wandb: 	K_tot: 192
wandb: 	batch_size: 8
wandb: 	convolution_type: ChebConv
wandb: 	epochs: 200
wandb: 	hidden_features: 128
wandb: 	n_layers_GNN: 6
wandb: 	n_layers_MLP: 4


best val at epoch,▁
val_loss,▁
best val at epoch,27
val_loss,0.01514


wandb: Agent Starting Run: wvnmiqy0 with config:
wandb: 	K_tot: 120
wandb: 	batch_size: 16
wandb: 	convolution_type: ChebConv
wandb: 	epochs: 200
wandb: 	hidden_features: 128
wandb: 	n_layers_GNN: 8
wandb: 	n_layers_MLP: 4


In [ ]:
#